In [1]:
import sqlite3

In [2]:
def init_db():
    conn = sqlite3.connect("tada.db")
    cursor = conn.cursor()

    cursor.execute("""
    CREATE TABLE Departments (
        dept_id INTEGER PRIMARY KEY AUTOINCREMENT,
        dept_name TEXT NOT NULL,
        hod_name TEXT
    );
    """)

    cursor.execute("""
    CREATE TABLE Students (
        student_id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        class INTEGER,
        dept_id INTEGER NOT NULL,
        FOREIGN KEY (dept_id)
            REFERENCES Departments (dept_id)
    );    
    """)

    cursor.execute("""
    CREATE TABLE Subjects (
        subject_id INTEGER PRIMARY KEY AUTOINCREMENT,
        subject_name TEXT,
        dept_id INTEGER NOT NULL,
        FOREIGN KEY (dept_id)
            REFERENCES departments(dept_id)
    );
    """)

    cursor.execute("""
    CREATE TABLE Marks (
        mark_id INTEGER PRIMARY KEY AUTOINCREMENT,
        student_id INTEGER NOT NULL,
        subject_id INTEGER NOT NULL,
        marks INTEGER,
        FOREIGN KEY (student_id)
            REFERENCES Students(student_id),
        FOREIGN KEY (subject_id)
            REFERENCES Subjects(subject_id)
    );
    """)

    departments = [
        ("Physics", "Big Johnny"),
        ("Mechnical", "John Boi"),
        ("Biology", "Little Johnny"),
        ("AeroSpace", "Johnny Girl")
    ]
    cursor.executemany("INSERT INTO Departments (dept_name, hod_name) VALUES (?, ?)", departments)

    studs = [
        ("Alice", 5, 1),
        ("Bob", 5, 2),
        ("Charlie", 8, 3),
        ("Diana", 9, 2),
        ("Eve", 4, 1),
        ("Test", 5, 4),
        ("Testy Boi", 7, 3),
        ("Testy Taste", 6, 4)
    ]
    cursor.executemany("INSERT INTO Students (name, class, dept_id) VALUES (?, ?, ?)", studs)

    subway = [
        ("Applied Maths I", 1),
        ("Applied Maths II", 1),
        ("ThemoDynamic I", 2),
        ("Internal Combustion Engine" , 2),
        ("Micro Biology", 3),
        ("Macro Biology", 3),
        ("Black Hole", 4),
        ("Super Nova", 4),
    ]
    cursor.executemany("INSERT INTO Subjects (subject_name, dept_id) VALUES (?, ?)", subway)

    markit = [
        (1,1,99),
        (1,2,90),
        (1,3,91),
        (1,4,92),
        (1,5,94),
        (1,6,95),
        (1,7,92),
        (1,8,30),

        (2,1,43),
        (2,2,65),
        (2,3,65),
        (2,4,44),
        (2,5,23),
        (2,6,43),
        (2,7,66),
        (2,8,98),

        (3,1,34),
        (3,2,54),
        (3,3,76),
        (3,4,87),
        (3,5,98),
        (3,6,67),
        (3,7,23),
        (3,8,54),

        (4,1,65),
        (4,2,43),
        (4,3,78),
        (4,4,98),
        (4,5,23),
        (4,6,43),
        (4,7,54),
        (4,8,64),

        (5,1,43),
        (5,2,55),
        (5,3,66),
        (5,4,77),
        (5,5,34),
        (5,6,54),
        (5,7,88),
        (5,8,33),

        (6,1,32),
        (6,2,90),
        (6,3,91),
        (6,4,92),
        (6,5,94),
        (6,6,95),
        (6,7,92),
        (6,8,93),

        (7,1,99),
        (7,2,90),
        (7,3,91),
        (7,4,92),
        (7,5,94),
        (7,6,95),
        (7,7,92),
        (7,8,93),

        (8,1,99),
        (8,2,92),
        (8,3,22),
        (8,4,11),
        (8,5,44),
        (8,6,55),
        (8,7,41),
        (8,8,78),
    ]
    cursor.executemany("INSERT INTO Marks(student_id,subject_id,marks) VALUES (?,?,?)",markit)

    conn.commit()
    return conn



In [3]:
conn = init_db()

#### Let's Go, The Create Methods

In [4]:
def add_student(conn, name, clasz, dept_id):
    conn.execute("INSERT INTO Students(name, class, dept_id) VALUES (?, ?, ?)", (name, clasz, dept_id))
    conn.commit()

def add_subject(conn, subject_name, dept_id):
    conn.execute("INSERT INTO Subjects(subject_name, dept_id) VALUES (?, ?)", (subject_name, dept_id))
    conn.commit()

def add_marks(conn, student_id, subject_id, marks):
    conn.execute("INSERT INTO Marks(student_id,subject_id,marks) VALUES (?,?,?)", (student_id, subject_id, marks))
    conn.commit()

#### Alright, Show Time~~~~( 0 _ 0 )~~~~

In [5]:
def printy(rows):
    for row in rows:
        print(" | ".join(f"{str(row)}" for row in row))

In [6]:
def show_students(conn):
    rez = conn.execute("""Select st.name, st.class, d.dept_name, d.hod_name 
        from Students st
        JOIN Departments d ON st.dept_id = d.dept_id    
    """).fetchall()
    printy(rez)

def show_subjects(conn):
    rez = conn.execute("""Select sb.subject_name, d.dept_name, d.hod_name 
        from Subjects sb
        JOIN Departments d ON sb.dept_id = d.dept_id    
    """).fetchall()
    printy(rez)
def show_marks(conn):
    rez = conn.execute("""Select st.name, sb.subject_name, mk.marks
        from Marks as mk
        JOIN Students st ON st.student_id = mk.student_id
        JOIN Subjects sb ON sb.subject_id = mk.subject_id    
    """).fetchall()
    printy(rez)
    pass

#### Advanced Queries - THE REPORTS~~~~( ^ _ 0 )~~~~

In [7]:
def student_report(conn, student_id):
    query = """
    SELECT st.name, sb.subject_name, mk.marks
    FROM Marks AS mk
    JOIN Students st ON st.student_id = mk.student_id
    JOIN Subjects sb ON sb.subject_id = mk.subject_id
    WHERE mk.student_id = ?
    """
    student_record = conn.execute(query, (student_id,)).fetchall()
    if not student_record:
        print(f"No records found for student ID {student_id}")
        return

    student_name = student_record[0][0]
    
    print(f"Student Report for: {student_name} (ID: {student_id})")
    print("-"*40)
    print(f"{'Subject':30} {'Marks'}")
    print("-"*40)
    
    total_marks = 0
    num_subjects = len(student_record)
    passed = True
    
    for name, subject, marks in student_record:
        print(f"{subject:30} {marks}")
        total_marks += marks
        if marks < 65:
            passed = False
    
    avg_marks = total_marks / num_subjects
    grade = get_grade(avg_marks)
    status = "Pass" if passed else "Fail"

    print("-"*40)
    print(f"Total Marks: {total_marks}")
    print(f"Average Marks: {avg_marks:.2f}")
    print(f"Grade: {grade}")
    print(f"Result: {status}")
    print("-"*40)


def get_grade(avg):
    grade_dict = {
        (90, 100): "A",
        (80, 89): "B",
        (70, 79): "C",
        (60, 69): "D",
        (0, 59): "F"
    }
    for (low, high), grade in grade_dict.items():
        if low <= avg <= high:
            return grade
    return "N/A"
student_report(conn,1)

Student Report for: Alice (ID: 1)
----------------------------------------
Subject                        Marks
----------------------------------------
Applied Maths I                99
Applied Maths II               90
ThemoDynamic I                 91
Internal Combustion Engine     92
Micro Biology                  94
Macro Biology                  95
Black Hole                     92
Super Nova                     30
----------------------------------------
Total Marks: 683
Average Marks: 85.38
Grade: B
Result: Fail
----------------------------------------


In [ ]:
def toppers(conn):
    cursor = conn.cursor()
    query_subject_toppers = """
    SELECT sb.subject_name, st.name, mk.marks
    FROM Marks mk
    JOIN Subjects sb ON mk.subject_id = sb.subject_id
    JOIN Students st ON mk.student_id = st.student_id
    WHERE (mk.subject_id, mk.marks) IN (
        SELECT subject_id, MAX(marks)
        FROM Marks
        GROUP BY subject_id
    )
    ORDER BY sb.subject_name;
    """
    subject_toppers = cursor.execute(query_subject_toppers).fetchall()
    current_subject = None

    print("Topper in each subject:")
    print("-"*50)
    for subject, student, marks in subject_toppers:
        if subject != current_subject:
            print(f"\nSubject: {subject}")
            current_subject = subject
        print(f"  {student} - Marks: {marks}")
    
    print("\n" + "-"*50)
    print("Overall Topper(s):")
    print("-"*50)

    query_overall_toppers = """
    SELECT st.name, SUM(mk.marks) as total_marks
    FROM Students st
    JOIN Marks mk ON st.student_id = mk.student_id
    GROUP BY st.student_id
    HAVING total_marks = (
        SELECT MAX(total) FROM (
            SELECT SUM(marks) as total FROM Marks GROUP BY student_id
        )
    )
    """
    
    overall_toppers = cursor.execute(query_overall_toppers).fetchall()
    for student, total in overall_toppers:
        print(f"{student} - Total Marks: {total}")

In [9]:
def department_summary(conn):
    query = """
    SELECT d.dept_name, AVG(mk.marks) as avg_marks
    FROM Departments d
    JOIN Students st ON d.dept_id = st.dept_id
    JOIN Marks mk ON st.student_id = mk.student_id
    GROUP BY d.dept_id, d.dept_name
    ORDER BY d.dept_name;
    """
    results = conn.execute(query).fetchall()
    print("Department Average Marks:")
    print("-" * 40)
    for dept_name, avg_marks in results:
        print(f"{dept_name:20} {avg_marks:.2f}")

In [10]:
def unique_subjects(conn):
    rez = conn.execute("SELECT DISTINCT subject_name FROM Subjects;").fetchall()
    subjects_set = {row[0] for row in rez}
    return subjects_set

In [11]:
show_students(conn)

Alice | 5 | Physics | Big Johnny
Bob | 5 | Mechnical | John Boi
Charlie | 8 | Biology | Little Johnny
Diana | 9 | Mechnical | John Boi
Eve | 4 | Physics | Big Johnny
Test | 5 | AeroSpace | Johnny Girl
Testy Boi | 7 | Biology | Little Johnny
Testy Taste | 6 | AeroSpace | Johnny Girl


In [12]:
show_subjects(conn)

Applied Maths I | Physics | Big Johnny
Applied Maths II | Physics | Big Johnny
ThemoDynamic I | Mechnical | John Boi
Internal Combustion Engine | Mechnical | John Boi
Micro Biology | Biology | Little Johnny
Macro Biology | Biology | Little Johnny
Black Hole | AeroSpace | Johnny Girl
Super Nova | AeroSpace | Johnny Girl


In [13]:
show_marks(conn)

Alice | Applied Maths I | 99
Alice | Applied Maths II | 90
Alice | ThemoDynamic I | 91
Alice | Internal Combustion Engine | 92
Alice | Micro Biology | 94
Alice | Macro Biology | 95
Alice | Black Hole | 92
Alice | Super Nova | 30
Bob | Applied Maths I | 43
Bob | Applied Maths II | 65
Bob | ThemoDynamic I | 65
Bob | Internal Combustion Engine | 44
Bob | Micro Biology | 23
Bob | Macro Biology | 43
Bob | Black Hole | 66
Bob | Super Nova | 98
Charlie | Applied Maths I | 34
Charlie | Applied Maths II | 54
Charlie | ThemoDynamic I | 76
Charlie | Internal Combustion Engine | 87
Charlie | Micro Biology | 98
Charlie | Macro Biology | 67
Charlie | Black Hole | 23
Charlie | Super Nova | 54
Diana | Applied Maths I | 65
Diana | Applied Maths II | 43
Diana | ThemoDynamic I | 78
Diana | Internal Combustion Engine | 98
Diana | Micro Biology | 23
Diana | Macro Biology | 43
Diana | Black Hole | 54
Diana | Super Nova | 64
Eve | Applied Maths I | 43
Eve | Applied Maths II | 55
Eve | ThemoDynamic I | 66
Eve

In [14]:
student_report(conn,1)

Student Report for: Alice (ID: 1)
----------------------------------------
Subject                        Marks
----------------------------------------
Applied Maths I                99
Applied Maths II               90
ThemoDynamic I                 91
Internal Combustion Engine     92
Micro Biology                  94
Macro Biology                  95
Black Hole                     92
Super Nova                     30
----------------------------------------
Total Marks: 683
Average Marks: 85.38
Grade: B
Result: Fail
----------------------------------------


In [15]:
toppers(conn)

Topper in each subject:
--------------------------------------------------

Subject: Applied Maths I
  Alice - Marks: 99
  Testy Boi - Marks: 99
  Testy Taste - Marks: 99

Subject: Applied Maths II
  Testy Taste - Marks: 92

Subject: Black Hole
  Alice - Marks: 92
  Test - Marks: 92
  Testy Boi - Marks: 92

Subject: Internal Combustion Engine
  Diana - Marks: 98

Subject: Macro Biology
  Alice - Marks: 95
  Test - Marks: 95
  Testy Boi - Marks: 95

Subject: Micro Biology
  Charlie - Marks: 98

Subject: Super Nova
  Bob - Marks: 98

Subject: ThemoDynamic I
  Alice - Marks: 91
  Test - Marks: 91
  Testy Boi - Marks: 91

--------------------------------------------------
Overall Topper(s):
--------------------------------------------------
Testy Boi - Total Marks: 746


In [16]:
department_summary(conn)

Department Average Marks:
----------------------------------------
AeroSpace            70.06
Biology              77.44
Mechnical            57.19
Physics              70.81


In [17]:
unique_subjects(conn)

{'Applied Maths I',
 'Applied Maths II',
 'Black Hole',
 'Internal Combustion Engine',
 'Macro Biology',
 'Micro Biology',
 'Super Nova',
 'ThemoDynamic I'}